In [1]:
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from loader import generate_patches
generate_patches('data/train','data/train_32',256,32,0.2,0.5)

processing data/train/turbulence/turbulence2
processing data/train/turbulence/turbulence3
processing data/train/turbulence/turbulence1
processing data/train/thermal/lakeSide
processing data/train/thermal/park
processing data/train/thermal/diningRoom
processing data/train/thermal/library
processing data/train/PTZ/zoomInZoomOut
processing data/train/PTZ/twoPositionPTZCam
processing data/train/PTZ/intermittentPan
processing data/train/dynamicBackground/overpass
processing data/train/dynamicBackground/fall
processing data/train/dynamicBackground/canoe
processing data/train/dynamicBackground/fountain02
processing data/train/dynamicBackground/fountain01
processing data/train/shadow/peopleInShade
processing data/train/shadow/cubicle
processing data/train/shadow/copyMachine
processing data/train/shadow/bungalows
processing data/train/shadow/busStation
processing data/train/nightVideos/streetCornerAtNight
processing data/train/nightVideos/tramStation
processing data/train/nightVideos/fluidHighw

# Data generator tests

In [7]:
from loader import PatchLoader
from dataset_tools import draw

def test_patch_loader(pl,original_data, dir, patch_size,o_size,seq_l,min_m,max_m,p_inx,inx):
    path = pl.patches[p_inx][inx]
    for i in range(10):
        im, mask, names, pos = pl.load_sample(p_inx, inx)
        if not (im is None):
            break
    o_imgs = []
    o_mask = []
    path = original_data+'/'.join(path[len(dir)+1:].split('/')[:2])
    for i in names:
        o_imgs.append(cv2.imread(path+'/input/in%s.jpg'%(i))[pos[1]:pos[1]+o_size,pos[0]:pos[0]+o_size])
        o_mask.append(cv2.imread(path+'/groundtruth/gt%s.png'%(i),0)[pos[1]:pos[1]+o_size,pos[0]:pos[0]+o_size])
    o_mask =np.array(o_mask)
    o_imgs = np.array(o_imgs)
    mask_diff = abs(np.array(o_mask.astype(np.float32)-mask.astype(np.float32)).sum())
    im_diff = abs(np.array(o_imgs[:20].astype(np.float32)-im[:20].astype(np.float32)).mean())
    print 'mask diff %f,img diff %f'%(mask_diff,im_diff)
    try:
        if(mask_diff > 0.001 or im_diff > 0.1 ):
            print mask_diff,im_diff
            draw(o_imgs,o_mask)
            raise ValueError('worng mask or img')
        if (im is None):
            raise ValueError('always None')
        if (im.shape != (seq_l, o_size, o_size, 3) or mask.shape != (seq_l, o_size, o_size)):
            print 'im_shape', im.shape, ',mask_shape', mask.shape
            raise ValueError('wrong shape')
        if not (min_m <= (mask[:, o_size // 2, o_size // 2] > 240).mean() <= max_m):
            print 'balanse', (mask[:, o_size // 2, o_size // 2] > 240).mean(), np.ceil(min_m * seq_l)
            raise ValueError('wrong balanse')
    except Exception as e:
        print 'max_m=%f,min_m=%f,seq_l=%d,o_size=%d,p_inx=%d,inx=%d,path=%s' % (
        max_m, min_m, seq_l, o_size, p_inx, inx, path)
        if not (im is None):
            draw(im, mask)
        raise e


def test_patch_loader_random(original_data, dir, patch_size, max_seq_l):
    o_size = int(np.random.randint(1, (patch_size - 1) // 2)) * 2 + 1
    seq_l = int(np.random.randint(5, np.sqrt(max_seq_l))) ** 2
    max_m = np.random.randint(1,seq_l)
    min_m = np.random.randint(0,max_m)
    min_m,max_m = min_m / float(seq_l), max_m / float(seq_l)
    pl = PatchLoader(dir, patch_size, seq_l,min_m,max_m , o_size)
    print(len(pl.patches))
    if(len(pl.patches) == 0):
        print 'no patches'
        print 'max_m=%f,min_m=%f,seq_l=%d,o_size=%d'%(max_m, min_m, seq_l, o_size)
    p_inx = np.random.randint(len(pl.patches))
    inx = np.random.randint(len(pl.patches[p_inx]))
    test_patch_loader(pl,original_data, dir, patch_size,o_size,seq_l,min_m,max_m,p_inx,inx)

for i in range(10):
    print i
    test_patch_loader_random('data/train/', 'data/train_32', 32, 256)

0
1674
mask diff 0.000000,img diff 0.030000
1
1192
mask diff 0.000000,img diff -0.006055
2
2651
mask diff 0.000000,img diff 0.041460
3
1424
mask diff 0.000000,img diff 0.017695
4
1332
mask diff 0.000000,img diff -0.239918
5
2651
mask diff 0.000000,img diff 0.011157
6
1821
mask diff 0.000000,img diff -0.009671
7
2476
mask diff 0.000000,img diff 0.005536
8
2583
mask diff 0.000000,img diff 0.005771
9
2376
mask diff 0.000000,img diff 0.027778


In [6]:
pl = PatchLoader('out',32,256,0.2,0.4,23,True)
%timeit pl.load_sample(0,0)

100 loops, best of 3: 6.8 ms per loop
